<a href="https://colab.research.google.com/github/mmovahed/UDLbook_ResNet/blob/main/Batch_Normalization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Run this if you're in a Colab to make a local copy of the MNIST 1D repository
!git clone https://github.com/greydanus/mnist1d

Cloning into 'mnist1d'...
remote: Enumerating objects: 204, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 204 (delta 47), reused 60 (delta 41), pack-reused 128
Receiving objects: 100% (204/204), 8.42 MiB | 13.50 MiB/s, done.
Resolving deltas: 100% (73/73), done.


In [3]:
import numpy as np
import os
import torch, torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from torch.optim.lr_scheduler import StepLR
import matplotlib.pyplot as plt
import mnist1d
import random

In [4]:
args = mnist1d.data.get_dataset_args()
data = mnist1d.data.get_dataset(args, path='./mnist1d_data.pkl', download=False, regenerate=False)

# The training and test input and outputs are in
# data['x'], data['y'], data['x_test'], and data['y_test']
print("Examples in training set: {}".format(len(data['y'])))
print("Examples in test set: {}".format(len(data['y_test'])))
print("Length of each example: {}".format(data['x'].shape[-1]))

Did or could not load data from ./mnist1d_data.pkl. Rebuilding dataset...
Examples in training set: 4000
Examples in test set: 1000
Length of each example: 40


In [5]:
# Load in the data
train_data_x = data['x'].transpose()
train_data_y = data['y']
val_data_x = data['x_test'].transpose()
val_data_y = data['y_test']
# Print out sizes
print("Train data: %d examples (columns), each of which has %d dimensions (rows)"%((train_data_x.shape[1],train_data_x.shape[0])))
print("Validation data: %d examples (columns), each of which has %d dimensions (rows)"%((val_data_x.shape[1],val_data_x.shape[0])))

Train data: 4000 examples (columns), each of which has 40 dimensions (rows)
Validation data: 1000 examples (columns), each of which has 40 dimensions (rows)


In [6]:
def print_variance(name, data):
  # First dimension(rows) is batch elements
  # Second dimension(columns) is neurons.
  np_data = data.detach().numpy()
  # Compute variance across neurons and average these variances over members of the batch
  neuron_variance = np.mean(np.var(np_data, axis=0))
  # Print out the name and the variance
  print("%s variance=%f"%(name,neuron_variance))

In [7]:
# He initialization of weights
def weights_init(layer_in):
  if isinstance(layer_in, nn.Linear):
    nn.init.kaiming_uniform_(layer_in.weight)
    layer_in.bias.data.fill_(0.0)

In [8]:
def run_one_step_of_model(model, x_train, y_train):
  # choose cross entropy loss function (equation 5.24 in the loss notes)
  loss_function = nn.CrossEntropyLoss()
  # construct SGD optimizer and initialize learning rate and momentum
  optimizer = torch.optim.SGD(model.parameters(), lr = 0.05, momentum=0.9)

  # load the data into a class that creates the batches
  data_loader = DataLoader(TensorDataset(x_train,y_train), batch_size=200, shuffle=True, worker_init_fn=np.random.seed(1))

  # Initialize model weights
  model.apply(weights_init)

  # Get a batch
  for i, data in enumerate(data_loader):
    # retrieve inputs and labels for this batch
    x_batch, y_batch = data
    # zero the parameter gradients
    optimizer.zero_grad()
    # forward pass -- calculate model output
    pred = model(x_batch)
    # compute the loss
    loss = loss_function(pred, y_batch)
    # backward pass
    loss.backward()
    # SGD update
    optimizer.step()
    # Break out of this loop -- we just want to see the first
    # iteration, but usually we would continue
    break

In [9]:
# convert training data to torch tensors
x_train = torch.tensor(train_data_x.transpose().astype('float32'))
y_train = torch.tensor(train_data_y.astype('long'))

In [10]:
# This is a simple residual model with 5 residual branches in a row
class ResidualNetwork(torch.nn.Module):
  def __init__(self, input_size, output_size, hidden_size=100):
    super(ResidualNetwork, self).__init__()
    self.linear1 = nn.Linear(input_size, hidden_size)
    self.linear2 = nn.Linear(hidden_size, hidden_size)
    self.linear3 = nn.Linear(hidden_size, hidden_size)
    self.linear4 = nn.Linear(hidden_size, hidden_size)
    self.linear5 = nn.Linear(hidden_size, hidden_size)
    self.linear6 = nn.Linear(hidden_size, hidden_size)
    self.linear7 = nn.Linear(hidden_size, output_size)

  def count_params(self):
    return sum([p.view(-1).shape[0] for p in self.parameters()])

  def forward(self, x):
    print_variance("Input",x)
    f = self.linear1(x)
    print_variance("First preactivation",f)
    res1 = f+ self.linear2(f.relu())
    print_variance("After first residual connection",res1)
    res2 = res1 + self.linear3(res1.relu())
    print_variance("After second residual connection",res2)
    res3 = res2 + self.linear4(res2.relu())
    print_variance("After third residual connection",res3)
    res4 = res3 + self.linear5(res3.relu())
    print_variance("After fourth residual connection",res4)
    res5 = res4 + self.linear6(res4.relu())
    print_variance("After fifth residual connection",res5)
    return self.linear7(res5)

In [11]:
# Define the model and run for one step
# Monitoring the variance at each point in the network
n_hidden = 100
n_input = 40
n_output = 10
model = ResidualNetwork(n_input, n_output, n_hidden)
run_one_step_of_model(model, x_train, y_train)

Input variance=0.966449
First preactivation variance=1.924398
After first residual connection variance=3.170713
After second residual connection variance=5.640253
After third residual connection variance=10.225904
After fourth residual connection variance=20.020832
After fifth residual connection variance=37.376015


In [12]:
import torch
import torch.nn as nn

class ResidualNetworkWithBatchNorm(torch.nn.Module):
    def __init__(self, input_size, output_size, hidden_size=100):
        super(ResidualNetworkWithBatchNorm, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.bn1 = nn.BatchNorm1d(hidden_size)  # Add batch normalization
        self.linear2 = nn.Linear(hidden_size, hidden_size)
        self.bn2 = nn.BatchNorm1d(hidden_size)  # Add batch normalization
        self.linear3 = nn.Linear(hidden_size, hidden_size)
        self.bn3 = nn.BatchNorm1d(hidden_size)  # Add batch normalization
        self.linear4 = nn.Linear(hidden_size, hidden_size)
        self.bn4 = nn.BatchNorm1d(hidden_size)  # Add batch normalization
        self.linear5 = nn.Linear(hidden_size, hidden_size)
        self.bn5 = nn.BatchNorm1d(hidden_size)  # Add batch normalization
        self.linear6 = nn.Linear(hidden_size, hidden_size)
        self.bn6 = nn.BatchNorm1d(hidden_size)  # Add batch normalization
        self.linear7 = nn.Linear(hidden_size, output_size)

    def count_params(self):
        return sum([p.view(-1).shape[0] for p in self.parameters()])

    def forward(self, x):
        print_variance("Input",x)
        f = self.linear1(x)
        f = self.bn1(f)  # Apply batch normalization
        f = f.relu()
        print_variance("First preactivation",f)
        res1 = f + self.linear2(f)
        res1 = self.bn2(res1)  # Apply batch normalization
        res1 = res1.relu()
        print_variance("After first residual connection",res1)
        res2 = res1 + self.linear3(res1)
        res2 = self.bn3(res2)  # Apply batch normalization
        res2 = res2.relu()
        print_variance("After second residual connection",res2)
        res3 = res2 + self.linear4(res2)
        res3 = self.bn4(res3)  # Apply batch normalization
        res3 = res3.relu()
        print_variance("After third residual connection",res3)
        res4 = res3 + self.linear5(res3)
        res4 = self.bn5(res4)  # Apply batch normalization
        res4 = res4.relu()
        print_variance("After fourth residual connection",res4)
        res5 = res4 + self.linear6(res4)
        res5 = self.bn6(res5)  # Apply batch normalization
        res5 = res5.relu()
        print_variance("After fifth residual connection",res5)
        return self.linear7(res5)

In [13]:
# Define the model
n_hidden = 100
n_input = 40
n_output = 10
model = ResidualNetworkWithBatchNorm(n_input, n_output, n_hidden)
run_one_step_of_model(model, x_train, y_train)

Input variance=0.913041
First preactivation variance=0.341479
After first residual connection variance=0.394049
After second residual connection variance=0.414613
After third residual connection variance=0.429958
After fourth residual connection variance=0.442648
After fifth residual connection variance=0.453828
